In [1]:
import keras

/home/wb329102/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [2]:
%%capture

%run ./path_manager.ipynb

In [3]:
import xlrd
import pandas as pd
import numpy as np

In [4]:
# https://stackoverflow.com/questions/7991209/identifying-excel-sheet-cell-color-code-using-xlrd-package
book = xlrd.open_workbook(os.path.join(get_corpus_path('IMF'), "imf_detected_acronyms_prototypes_OD - Copy.xls"), formatting_info=True)
sheets = book.sheet_names()
print(f"sheets are: {sheets}")

sheets are: ['imf_detected_acronyms_prototype', 'Selected_list']


In [ ]:
# Energy Information Agency?

In [5]:
# cell bgx == 13 (yellow highlight - positive samples)
# cell bgx == 64 (no highlight - negative samples or duplicates)
# column (0) 1 is the acronym

In [6]:
acronym_dataset = []

for index, sh in enumerate(sheets):
    sheet = book.sheet_by_index(index)
    print("Sheet:", sheet.name)
    if sheet.name != 'imf_detected_acronyms_prototype':
        continue
        
    rows, cols = sheet.nrows, sheet.ncols
    print("Number of rows: %s   Number of cols: %s" % (rows, cols))
    for row in range(rows):
        if row == 0:
            continue
        goto_next_row = False
        for col in range(cols):
            cell = sheet.cell(row, col)      
            # could get 'dump', 'value', 'xf_index'
            cell_value = cell.value
            xfx = sheet.cell_xf_index(row, col)
            xf = book.xf_list[xfx]
            bgx = xf.background.pattern_colour_index
            
            # print(f"row={row + 1}, col={col + 1}, val={cell_value}, bg={bgx}")

            if (col + 1) == 1:
                acr = cell_value
                if not acr:
                    goto_next_row = True
                    break

            elif (col + 1) % 2 != 0:
                acronym_dataset.append({
                    'acronym': acr,
                    'full_name': cell_value,
                    'is_valid': bgx == 13
                })
                
acronym_dataset = pd.DataFrame(acronym_dataset)

Sheet: imf_detected_acronyms_prototype
Number of rows: 3554   Number of cols: 12
Sheet: Selected_list


In [7]:
acronym_dataset.shape

(17710, 3)

# Remove unlabelled data

In [8]:
labeled_acronyms = acronym_dataset.groupby('acronym')['is_valid'].sum() > 0
labeled_acronyms = set(labeled_acronyms[labeled_acronyms].index)

In [9]:
working_dataset = acronym_dataset[acronym_dataset.acronym.isin(labeled_acronyms)]

# Remove duplicates in negative samples

In [10]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
hvec = HashingVectorizer(analyzer='char', ngram_range=(2, 3))

In [51]:
from nltk.metrics.distance import edit_distance

In [55]:
def get_valid_negative_samples(df, max_dissimilarity_thresh=2):
    if not df[False]:
        return []
    
    negatives = np.array(df[False])
    positives = df[True]
    true_negatives = set()
    
    for p in positives:
        for n in negatives:
            e = edit_distance(p, n)
            if e > max_dissimilarity_thresh:
                true_negatives.add(n)
                
#     neg_vec = hvec.transform(negatives)
#     sim = cosine_similarity(neg_vec, hvec.transform(positives))
#     print(sim)
#     sim = sim.max(axis=1) # Get maximum similarity, meaning, remove term regardless of which prototype it's a duplicate of.
    
#     true_negatives = negatives[np.where(sim < max_dissimilarity_thresh)]
    
    return list(true_negatives)

In [57]:
q = working_dataset.groupby(['acronym', 'is_valid']).apply(lambda x: x['full_name'].tolist()).unstack()

In [58]:
%%time
negative_samples = q.apply(get_valid_negative_samples, axis=1)

CPU times: user 3.62 s, sys: 21 ms, total: 3.64 s
Wall time: 17 s


In [59]:
full_valid_data = pd.DataFrame(q)
full_valid_data[False] = negative_samples

# Redundant but this is fast... :D
negatives = [{'acronym': k, 'full_name': v, 'is_valid': False} for k, vs in full_valid_data[False].items() for v in vs]
positives = [{'acronym': k, 'full_name': v, 'is_valid': True} for k, vs in full_valid_data[True].items() for v in vs]

full_valid_data = pd.DataFrame(positives + negatives)

# Features

1. Match in first letters
2. 

In [64]:
q.loc['WHO'][False]

['World Health Organi', 'Index (UNDP) Health', 'World Health Organiza']

In [65]:
get_valid_negative_samples(q.loc['WHO'])

['World Health Organi', 'Index (UNDP) Health', 'World Health Organiza']

In [66]:
full_valid_data[full_valid_data.acronym == 'WFP']

,acronym,full_name,is_valid
1090,WFP,World Food Program,True
3327,WFP,Simon Giampiero Lucarini,False
3328,WFP,World Food Progranune,False
3329,WFP,Working Family Payment,False


In [67]:
full_valid_data.head()

,acronym,full_name,is_valid
0,AAA,Analytical Advisory Activities,True
1,AAP,Assessment Action Plan,True
2,ABS,Australian Bureau Statistics,True
3,ACC,Anti Corruption Commission,True
4,ACE,Allowance Corporate Equity,True
